# 朝倉書店「医学への統計学」6章 相関係数と回帰直線に関する推測

例題をJuliaを使って解いてみる。

In [43]:
using Distributions
using HypothesisTests
using Plots

## 例題6.1

ある健康センターの、男性受診者53名のデータを用いて、Hb、Na、UA、Alb、T-Cholの5項目間の相関係数を計算した。HbとNa間の相関係数は0.015（データ数は43）、HbとUA間の相関係数は-0.533（データ数は35）であった。有意性の検定を行え。

In [13]:
# 統計量の関数を定義
function calc_t_stat(r, n, α)
    nume = r * √(n-2) 
    denom = √(1-r^2)
    t_stat = abs(nume/denom)
    upper = cquantile.(TDist(n-2), α/2)
    return t_stat, upper
end

calc_t_stat (generic function with 2 methods)

In [18]:
# HbとNa間
r1 = 0.015; n1 = 43
α = [0.05, 0.001]
t_stat1, upper1 = calc_t_stat(r1, n1, α)
println("HbとNa間の統計量: ", round(t_stat1, digits=3))
println("上側パーセント点（有意水準5%）: ", round(upper1[1], digits=3))
println("上側パーセント点（有意水準0.1%）: ", round(upper1[2], digits=3))

HbとNa間の統計量: 0.096
上側パーセント点（有意水準5%）: 2.02
上側パーセント点（有意水準0.1%）: 3.544


In [16]:
# HbとUA間
r2 = -0.533; n2 = 35
α = [0.05, 0.001]
t_stat2, upper2 = calc_t_stat(r2, n2, α)
println("HbとUA間の統計量: ", round(t_stat2, digits=3))
println("上側パーセント点（有意水準5%）: ", round(upper2[1], digits=3))
println("上側パーセント点（有意水準0.1%）: ", round(upper2[2], digits=3))

HbとUA間の統計量: 3.619
上側パーセント点（有意水準5%）: 2.035
上側パーセント点（有意水準0.1%）: 3.611


よって、HbとUA間は有意水準0.1%で有意であった。

## 例題6.2

例題6.1のHbとUA間の相関係数の95%信頼区間を求めよ。

In [26]:
# 信頼区間の関数を定義
function confidence_interval(r, n, α)
    a = 0.5 * log((1+r)/(1-r)) - 1/√(n-3) * cquantile(Normal(0, 1), α/2)
    b = 0.5 * log((1+r)/(1-r)) + 1/√(n-3) * cquantile(Normal(0, 1), α/2)
    lower = (exp(2*a)-1) / (exp(2*a) + 1)
    upper = (exp(2*b)-1) / (exp(2*b) + 1)
    return lower, upper
end

confidence_interval (generic function with 1 method)

In [30]:
r = -0.533; n = 35; α = 0.05
lower, upper = confidence_interval(r, n, α)
println(round(lower, digits=3), " ≤ ρ ≤ ", round(upper, digits=3))

-0.736 ≤ ρ ≤ -0.243


## 例題6.3

例題6.1で今度は女性の受診者だけで相関係数を求めたところ、UAとAlbの相関係数は$r_B$=-0.0971($n_B$=24)と得られた。この二つの項目間の男性の相関係数は$r_A$=-0.459($n_A$=30)となっている。性差は認められるか。

In [38]:
# 統計量の関数を定義
function calc_t_stat(r1, r2, n1, n2, α)
    nume = 0.5*log((1+r1)/(1-r1)) - 0.5*log((1+r2)/(1-r2))
    denom = √((1/(n1-3)) + (1/(n2-3)))
    t_stat = abs(nume/denom)
    upper = cquantile(Normal(0, 1), α/2)
    println("統計量: ", round(t_stat, digits=3))
    println("上側パーセント点 (α=$α): ", round(upper, digits=3))
end

calc_t_stat (generic function with 3 methods)

In [39]:
r1 = -0.459; r2 = -0.0971; n1 = 30; n2 = 24; α = 0.05
calc_t_stat(r1, r2, n1, n2, α)

統計量: 1.37
上側パーセント点 (α=0.05): 1.96


よって、性差は有意水準5%で認められない。

## 例題6.4

いくつかの地域での、3〜4歳児のむし歯保有率とむし歯罹患型の関連を調査したところ、ある型のむし歯罹患率との関連データを得た。このデータから相関が認められるか。

In [44]:
むし歯保有率 = [33, 24, 30, 50, 42, 15, 5, 56, 13, 45, 44, 21, 8, 31, 27, 40]
むし歯罹患率 = [20, 4, 50, 10, 78, 3, 2, 4, 6, 56, 42, 5, 5, 71, 9, 6];

In [53]:
# 統計量の関数を定義
using StatsBase

function calc_t_stat(d1, d2, α)    
    r_s = corspearman(d1, d2)  # StatsBase.ji
    n = length(d1)
    nume = r_s * √(n-2)
    denom = √(1-r_s^2)
    t_stat = abs(nume/denom)
    upper = cquantile(TDist(n-2), α/2)
    println("Spearmanの順位相関係数: ", round(r_s, digits=3))
    println("統計量: ", round(t_stat, digits=3))
    println("上側パーセント点 (α=$α): ", round(upper, digits=3))
end

calc_t_stat (generic function with 3 methods)

In [54]:
calc_t_stat(むし歯保有率, むし歯罹患率, 0.05)

Spearmanの順位相関係数: 0.529
統計量: 2.333
上側パーセント点 (α=0.05): 2.145


よって、有意水準5%で相関が認められる。